In [2]:
import os
import keras

os.environ['CUDA_VISIBLE_DEVICES']='0'

Using TensorFlow backend.


In [3]:
train,test=keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
def preproc_x(x,flatten=True):
  x=x/255.
  if flatten:
    x=x.reshape((len(x),-1))
  return x

def preproc_y(y,num_classes=10):
  if y.shape[-1]==1:
    y=keras.utils.to_categorical(y,num_classes)
  return y

In [0]:
x_train,y_train=train
x_test,y_test=test

x_train=preproc_x(x_train)
x_test=preproc_x(x_test)

y_train=preproc_y(y_train)
y_test=preproc_y(y_test)

In [0]:
from keras.regularizers import l1,l2,l1_l2

def build_mlp(input_shape,output_units=10,num_neurons=[512,256,128],l2_ratio=1e-4):
  input_layer=keras.layers.Input(input_shape)

  for i,n_units in enumerate(num_neurons):
    if i==0:
      x=keras.layers.Dense(units=n_units,activation='relu',name='hidden_layer'+str(i+1),kernel_regularizer=l2(l2_ratio))(input_layer)
    else:
      x=keras.layers.Dense(units=n_units,activation='relu',name='hidden_layer'+str(i+1),kernel_regularizer=l2(l2_ratio))(x)
  out=keras.layers.Dense(units=output_units,activation='softmax',name='output')(x)

  model=keras.models.Model(inputs=[input_layer],outputs=[out])
  return model


In [0]:
LEARNING_RATE=1e-3
EPOCHS=50
BATCH_SIZE=256
MOMENTUM=0.95
L2_EXP=[1e-2,1e-4,1e-8,1e-12]

In [0]:
results={}

for regulizer_ratio in L2_EXP:
  keras.backend.clear_session()
  print('Experiment with Regulizer = %.6f'%(regulizer_ratio))
  model=build_mlp(input_shape=x_train.shape[1:],l2_ratio=regulizer_ratio)
  model.summary()
  optimizer=keras.optimizers.SGD(lr=LEARNING_RATE,nesterov=True,momentum=MOMENTUM)
  model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optimizer)

  model.fit(x_train,y_train,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_data=(x_test,y_test),shuffle=True)

  train_loss=model.history.history['loss']
  valid_loss=model.history.history['val_loss']
  train_acc=model.history.history['acc']
  valid_acc=model.history.history['val_acc']

  exp_name_tag='exp-l2-%s'%str(regulizer_ratio)
  results[exp_name_tag]={'train-loss':train_loss,'valid-loss':valid_loss,'train-acc':train_acc,'valid-acc':valid_acc}





Experiment with Regulizer = 0.010000


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broa

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

color_bar=['r','g','b','y','m','k']
plt.figure(figsize=(8,6))

for i, cond in enumerate(results.keys()):
  plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'],'-',label=cond,color=color_bar[i])
  plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'],'--',label=cond,color=color_bar[i])
plt.title('Loss')
plt.ylim([0,5])
plt.legend()
plt.show()

for i, cond in enumerate(results.keys()):
  plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'],'-',label=cond,color=color_bar[i])
  plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'],'--',label=cond,color=color_bar[i])
plt.title('Loss')
plt.ylim([0,5])
plt.legend()
plt.show()